In [9]:

from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from utils import ProgramDataset, make_max_length
print("Starting kernel")
model_run_name = "l2cfg"
param_size = "125M"
model_dir = "../../../output/gpt-results-%s-%s" % (param_size, model_run_name)


Starting kernel


In [13]:
from collections.abc import Iterable, Iterator
from functools import reduce
def canBeList(x):
    return isinstance(x, Iterable)

def expand_iters(e):
    if(canBeList(e)): # Expand it out.
        return [expand_iters(ele) for ele in e]
    return e
def mapt(f, x):
    if(len(x) == 0):
        return f(x)
    # TODO: this
    return x
def cons(a, b):
    blist = canBeList(b)
    alist = canBeList(a)
    al = expand_iters(a) if alist else [a]
    bl = expand_iters(b) if blist else [b]
    return al + bl

foldl = lambda f, acc, xs: reduce(f, xs, acc)
foldr = lambda f, acc, xs: reduce(lambda x, y: f(y, x), xs[::-1], acc)
def recl(f, e, x):
    if(len(x) == 0):
        return e
    return f(x[0], x[1:])



In [14]:

# Now, make the outputs for us to evaluate:
fine_model = GPTNeoForCausalLM.from_pretrained(model_dir).cuda()
fine_tokenizer = GPT2Tokenizer.from_pretrained(model_dir, 
    bos_token="<|startoftext|>",
    eos_token="<|endoftext|>",
    pad_token="<|pad|>"
)

def do_gpt(seed_text):
    input_tensor = fine_tokenizer(seed_text, return_tensors="pt").input_ids.cuda()
    outputs = fine_model.generate(
        input_tensor, 
        max_length=2048,  
        # num_return_sequences=5,
        # no_repeat_ngram_size=2,
        # repetition_penalty=1.5,
        top_p=0.95,
        temperature=0.05,
        do_sample=True,
        top_k=50,
        # early_stopping=False
    )
    output = outputs[0]
    out_str = fine_tokenizer.decode(output) 
    out_prog = out_str.split('Program:')[1].replace("<|endoftext|>", "")
    return out_prog

In [12]:
longstr = """
Examples:
Inputs: 
[9, 14, -12, 7, 8]
Output: 
[9, 14, -12, 7, 8, 9, 14, -12, 7, 8]
Inputs: 
[-14, 3]
Output: 
[-14, 3, -14, 3]
Inputs: 
[-9, 13, -19, -4, 14]
Output: 
[-9, 13, -19, -4, 14, -9, 13, -19, -4, 14]
Inputs: 
[6, -14]
Output: 
[6, -14, 6, -14]
Inputs: 
[5, -7]
Output: 
[5, -7, 5, -7]
Inputs: 
[-14, 20, -11]
Output: 
[-14, 20, -11, -14, 20, -11]
Inputs: 
[17]
Output: 
[17, 17]

Program:
"""

# Original program: 
"""

Program: 
<prog> -> "lambda" <lamargs> ":" <stmt> ATTRS: {retType=[[int]], y_is_decl=true}
	"lambda" ->  ATTRS: {}
	<lamargs> -> <varInit> ATTRS: {length=1, y_is_decl=true}
		<varInit> -> <lowercaseAscii> ATTRS: {y_is_decl=true}
			<lowercaseAscii> -> "y" ATTRS: {chosenSymbol=y, retType=[[int]]}
				"y" ->  ATTRS: {}
	":" ->  ATTRS: {}
	<stmt> -> "cons" "(" <declared> "," <declared> ")" ATTRS: {retType=[[int]], y_is_decl=true}
		"cons" ->  ATTRS: {}
		"(" ->  ATTRS: {}
		<declared> -> <lowercaseAscii> ATTRS: {chosenSymbol=y, retType=[[int]], y_is_decl=true}
			<lowercaseAscii> -> "y" ATTRS: {chosenSymbol=y, retType=[[int]], y_is_decl=true}
				"y" ->  ATTRS: {}
		"," ->  ATTRS: {}
		<declared> -> <lowercaseAscii> ATTRS: {chosenSymbol=y, retType=[[int]], y_is_decl=true}
			<lowercaseAscii> -> "y" ATTRS: {chosenSymbol=y, retType=[[int]], y_is_decl=true}
				"y" ->  ATTRS: {}
		")" ->  ATTRS: {}
		"""

out = do_gpt(longstr)
print(out)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: list index out of range

In [ ]:
cartp = """
Inputs: 
[[1], [2, 3]]
Outputs:
[[1, 2], [1, 3]]
Inputs: 
[[]]
Outputs:
[]
Inputs: 
[[1, 2, 3], [4, 5]]
Output:
[[1, 4], [1, 5], [2, 4], [2, 5], [3, 4], [3, 5]]
Inputs: 
[[1], [2, 3], [4, 5]]
Output:
[[1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5]]

Program:
"""

    func = eval(prog_py)
    print(expand_iters(func([[1, 2, 3], [4, 5]])))



IndentationError: unexpected indent (1530148321.py, line 22)